### 1. Init config

In [1]:
from google.cloud import bigquery
import os
import pandas as pd
import datetime

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

client = bigquery.Client()

project_id = "zapart-data-vlille"
dataset_id = "vlille_dataset"

### 2. Stations query - dim table

In [3]:
table_name = "stations"

table_ref = client.get_table(f'{project_id}.{dataset_id}.{table_name}')
schema = table_ref.schema

for field in schema:
    print(field.name, field.field_type)


id INTEGER
nom STRING
adresse STRING
commune STRING
type STRING
latitude FLOAT
longitude FLOAT


In [4]:
sql = f"""
SELECT *
FROM `{project_id}.{dataset_id}.stations`
"""

df_stations = client.query(sql).to_dataframe()

In [5]:
df_stations.head()

id               nom                    adresse            commune  \
0  136      RUE CHAMBORD               RUE CHAMBORD  VILLENEUVE D'ASCQ   
1  270     PAVÉ DE LILLE              Rue Jean Bart          WASQUEHAL   
2   35           LECLERC  24 PLACE MARECHAL LECLERC              LILLE   
3   44  GAMBETTA UTRECHT      199 RUE LEON GAMBETTA              LILLE   
4   11    NOUVEAU SIECLE              10 RUE DE PAS              LILLE   

       type   latitude  longitude  
0  SANS TPE  50.629720   3.133873  
1  AVEC TPE  50.664211   3.130098  
2  AVEC TPE  50.628990   3.043307  
3  AVEC TPE  50.629063   3.053711  
4  AVEC TPE  50.637340   3.060977

In [6]:
# add a column 'geo' [latitude, longitude]
df_stations['geo'] = df_stations.apply(lambda row: str(row['latitude']) + "," + str(row['longitude']), axis=1)
df_stations.head()

id               nom                    adresse            commune  \
0  136      RUE CHAMBORD               RUE CHAMBORD  VILLENEUVE D'ASCQ   
1  270     PAVÉ DE LILLE              Rue Jean Bart          WASQUEHAL   
2   35           LECLERC  24 PLACE MARECHAL LECLERC              LILLE   
3   44  GAMBETTA UTRECHT      199 RUE LEON GAMBETTA              LILLE   
4   11    NOUVEAU SIECLE              10 RUE DE PAS              LILLE   

       type   latitude  longitude                 geo  
0  SANS TPE  50.629720   3.133873   50.62972,3.133873  
1  AVEC TPE  50.664211   3.130098  50.664211,3.130098  
2  AVEC TPE  50.628990   3.043307   50.62899,3.043307  
3  AVEC TPE  50.629063   3.053711  50.629063,3.053711  
4  AVEC TPE  50.637340   3.060977   50.63734,3.060977

In [7]:
# save dataframe to csv
df_stations.to_csv('stations.csv', index=False)

### 3. Records query - fact table

In [3]:
table_name = "records"

table_ref = client.get_table(f'{project_id}.{dataset_id}.{table_name}')
schema = table_ref.schema

for field in schema:
    print(field.name, field.field_type)


station_id INTEGER
etat STRING
nb_velos_dispo INTEGER
nb_places_dispo INTEGER
etat_connexion STRING
derniere_maj TIMESTAMP
record_timestamp TIMESTAMP


In [10]:
sql = f"""
SELECT *
FROM 
    `{project_id}.{dataset_id}.{table_name}`
WHERE
    station_id = 25
    AND DATE(record_timestamp) = "2023-11-13"
"""

df_records = client.query(sql).to_dataframe()

# remove duplicate rows
df_records = df_records.drop_duplicates()
df_records.sort_values(by=['record_timestamp'], inplace=True)
df_records.head()

station_id        etat  nb_velos_dispo  nb_places_dispo etat_connexion  \
231          25  EN SERVICE               0               32       CONNECTÉ   
232          25  EN SERVICE               0               32       CONNECTÉ   
233          25  EN SERVICE               0               32       CONNECTÉ   
234          25  EN SERVICE               0               32       CONNECTÉ   
235          25  EN SERVICE               0               32       CONNECTÉ   

                 derniere_maj                 record_timestamp  
231 2023-11-12 23:56:10+00:00 2023-11-13 00:00:06.624000+00:00  
232 2023-11-12 23:56:10+00:00 2023-11-13 00:01:01.149000+00:00  
233 2023-11-12 23:56:10+00:00 2023-11-13 00:02:01.485000+00:00  
234 2023-11-12 23:56:10+00:00 2023-11-13 00:03:01.494000+00:00  
235 2023-11-12 23:56:10+00:00 2023-11-13 00:04:00.899000+00:00

In [24]:
sql = f"""
SELECT station_id, nom, nb_velos_dispo, nb_places_dispo, record_timestamp, adresse, commune, latitude, longitude, type, etat, etat_connexion, derniere_maj
FROM 
    `{project_id}.{dataset_id}.stations`, 
    `{project_id}.{dataset_id}.records`
WHERE
    `{project_id}.{dataset_id}.stations`.id = `{project_id}.{dataset_id}.records`.station_id
    AND DATE(record_timestamp) = "2023-11-13"
"""

df_full = client.query(sql).to_dataframe()

In [25]:
df_full.head()

station_id                    nom  nb_velos_dispo  nb_places_dispo  \
0           2  UNIVERSITE CATHOLIQUE              31                0   
1           2  UNIVERSITE CATHOLIQUE              31                0   
2           2  UNIVERSITE CATHOLIQUE              31                0   
3           2  UNIVERSITE CATHOLIQUE              31                0   
4           2  UNIVERSITE CATHOLIQUE              31                0   

                  record_timestamp                adresse commune   latitude  \
0 2023-11-13 08:57:01.256000+00:00  RUE DU PORT BD VAUBAN   LILLE  50.632233   
1 2023-11-13 08:58:03.077000+00:00  RUE DU PORT BD VAUBAN   LILLE  50.632233   
2 2023-11-13 08:59:00.917000+00:00  RUE DU PORT BD VAUBAN   LILLE  50.632233   
3 2023-11-13 09:00:45.806000+00:00  RUE DU PORT BD VAUBAN   LILLE  50.632233   
4 2023-11-13 09:00:45.806000+00:00  RUE DU PORT BD VAUBAN   LILLE  50.632233   

   longitude      type        etat etat_connexion              derniere_maj  
0   3.046134  AVEC TPE  EN SERVICE       CONNECTÉ 2023-11-13 08:55:08+00:00  
1   3.046134  AVEC TPE  EN SERVICE       CONNECTÉ 2023-11-13 08:56:08+00:00  
2   3.046134  AVEC TPE  EN SERVICE       CONNECTÉ 2023-11-13 08:57:09+00:00  
3   3.046134  AVEC TPE  EN SERVICE       CONNECTÉ 2023-11-13 08:57:09+00:00  
4   3.046134  AVEC TPE  EN SERVICE       CONNECTÉ 2023-11-13 08:57:09+00:00

In [26]:
df_full.shape

(415582, 13)

In [27]:
df_full['geo'] = df_full.apply(lambda row: str(row['latitude']) + "," + str(row['longitude']), axis=1)

In [18]:
df_light = df_full[['station_id', 'nb_velos_dispo', 'nb_places_dispo', 'latitude', 'longitude', 'record_timestamp']]

In [19]:
df_light.head()

station_id  nb_velos_dispo  nb_places_dispo   latitude  longitude  \
0           2              31                0  50.632233   3.046134   
1           2              31                0  50.632233   3.046134   
2           2              31                0  50.632233   3.046134   
3           2              31                0  50.632233   3.046134   
4           2              31                0  50.632233   3.046134   

                  record_timestamp  
0 2023-11-13 08:57:01.256000+00:00  
1 2023-11-13 08:58:03.077000+00:00  
2 2023-11-13 08:59:00.917000+00:00  
3 2023-11-13 09:00:45.806000+00:00  
4 2023-11-13 09:00:45.806000+00:00

In [22]:
df_light.to_csv('dataset_light.csv', index=False)

In [28]:
df_full.to_csv('datataset_full.csv', index=False)